<a href="https://colab.research.google.com/github/gouravbeyondBGCM/BeyondBGCM/blob/master/text_summarize_13_feb_21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords
from collections import defaultdict
from string import punctuation
from heapq import nlargest
import urllib.request
from bs4 import BeautifulSoup

nltk.download('stopwords')
nltk.download('punkt')

class FrequencySummarizer:
  def __init__(self, min_cut=0.1, max_cut=0.9):
    """
     Initilize the text summarizer.
     Words that have a frequency term lower than min_cut 
     or higer than max_cut will be ignored.
    """
    self._min_cut = min_cut
    self._max_cut = max_cut 
    self._stopwords = set(stopwords.words('english') + list(punctuation))

  def _compute_frequencies(self, word_sent):
    """ 
      Compute the frequency of each of word.
      Input: 
       word_sent, a list of sentences already tokenized.
      Output: 
       freq, a dictionary where freq[w] is the frequency of w.
    """
    print ('----inside _compute_frequencies--1')
    freq = defaultdict(int)
    print ('----inside _compute_frequencies--1.1')
    for s in word_sent:
      for word in s:
        if word not in self._stopwords:
          freq[word] += 1
    # frequencies normalization and fitering
    #error in for loop
    print ('----inside _compute_frequencies--2')
    m = float(max(freq.values()))
    print ('-----Length of Array---' + str(len(freq)))
    for w in list(freq):
      freq[w] = freq[w]/m
      if freq[w] >= self._max_cut or freq[w] <= self._min_cut:
        del freq[w]
        #print ('----inside _compute_frequencies--3')
    return freq

  def summarize(self, text, n):
    """
      Return a list of n sentences 
      which represent the summary of text.
    """
    print ('----inside summarize--')
    sents = sent_tokenize(text)
    assert n <= len(sents)
    print ('----inside summarize--1')
    word_sent = [word_tokenize(s.lower()) for s in sents]
    print ('----inside summarize--1.1')
    print(word_sent)
    self._freq = self._compute_frequencies(word_sent)
    print ('----inside summarize--1.2')
    ranking = defaultdict(int)
    print ('----inside summarize--2')
    for i,sent in enumerate(word_sent):
      for w in sent:
        if w in self._freq:
          ranking[i] += self._freq[w]
    sents_idx = self._rank(ranking, n)    
    print ('----inside summarize--3')
    return [sents[j] for j in sents_idx]

  def _rank(self, ranking, n):
    """ return the first n sentences with highest ranking """
    return nlargest(n, ranking, key=ranking.get)
 

In [52]:
import urllib.request
from bs4 import BeautifulSoup
def get_only_text(url):
    """ 
      return the title and the text of the article at the specified url
    """
    page = urllib.request.urlopen(url).read().decode('utf8')
    soup = BeautifulSoup(page)
    text = ' '.join(map(lambda p: p.text, soup.find_all('p')))
    return soup.title.text, text

  

In [53]:
feed_xml = urllib.request.urlopen('http://feeds.bbci.co.uk/news/rss.xml').read()
feed = BeautifulSoup(feed_xml.decode('utf8'))
to_summarize = list(map(lambda p: p.text, feed.find_all('guid')))

fs = FrequencySummarizer()
for article_url in to_summarize[:5]:
    title, text = get_only_text(article_url)
    print ('----------------------------------')
    print (title)
    print(text)
    summary = fs.summarize(text, 8)
    for s in summary :
        print ('*',s)

----------------------------------
Covid-19: Boris Johnson plans to reopen shops and gyms in England on 12 April - BBC News
Shops, hairdressers, gyms and outdoor hospitality will reopen on 12 April in England if strict conditions are met, under plans being set out by the PM. Up to six people from separate households could be able to meet in beer gardens from that date. The new four-step plan to ease lockdown could see all legal limits on social contact lifted by 21 June. It requires four tests on vaccines, infection rates and new coronavirus variants to be met at each stage. Prime Minister Boris Johnson told MPs the plan aimed to be "cautious but irreversible" and at every stage decisions would be led by "data not dates". But he warned there was "no credible route to a zero-Covid Britain nor indeed a zero-Covid world". It comes as the first data on the UK's coronavirus vaccine rollout suggests it is having a "spectacular" impact on stopping serious illness. As part of the first step of

In [54]:
 import nltk
 nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [55]:
pip install urllib.request

ERROR: Could not find a version that satisfies the requirement urllib.request (from versions: none)
ERROR: No matching distribution found for urllib.request
